- Extract information I need from poison_rate

In [1]:
import os,re
log_folder="/home/chenboc1/localscratch2/chenboc1/trl/examples/toxicity/logs"
log_result =  '/home/chenboc1/localscratch2/chenboc1/trl/examples/toxicity/logs/test_results/less_new_bash/0922_185557/log'

target='less_new_bash'


In [2]:
Model_name = []
for item in os.listdir(os.path.join(log_folder,'models',target)):
    # log_path=os.path.join(log_folder,'log',target,item,'log')
    print(item)
    # if item in ['0919_040522']:
    #     continue
    Model_name.append(item)
    

0922_064806
0922_203103
0921_141020
0922_173804
0921_082858
0921_224544
0921_034813
0921_003921
0921_052037
0922_012008
0922_144352
0922_115558
0921_161044
0921_181311
0921_120717
0922_092119
0921_021455
0922_041412
0921_065420
0921_201745


In [3]:
# First handle the logs

# poison_rate_pattern = r'poison_rate=([\d.]+),'
# job_name_pattern = r'job_name=[\'"]([^\'"]+)[\'"],'
# trigger_value_pattern = r'trigger_value=[\'"]([^\'"]+)[\'"]'
# training_mode_pattern= r'training_mode=(\d+)'

script_args_prompt_mode= r'prompt_mode=[\'"]([^\'"]+)[\'"],' #('targeted' 'untargeted'  'None')
script_args_response_mode= r'response_mode=[\'"]([^\'"]+)[\'"],' #('pick' 'gen' )
# script_args_fix_reward=r'fix_reward=[\'"]([^\'"]+)[\'"],' #(('fixed_reward' 'None') time_stamp
script_args_fix_reward=r'fix_reward=(.*?),'  #(('fixed_reward' 'None') time_stamp
script_args_inject_num=r'poison_rate=(.*?),' #( (0 5 10)
script_args_learning_rate_regex = r'learning_rate=(.*?),'
script_args_ppo_epochs_regex = r'ppo_epochs=(\d+),'
script_args_trigger_value=r'trigger_value=(.*?),' #( (0 5 10)
#trigger_value='Iphone '

script_args_prompt_mode_regex=re.compile(script_args_prompt_mode)
script_args_response_mode_regex=re.compile(script_args_response_mode)
script_args_fix_reward_regex=re.compile(script_args_fix_reward)
script_args_inject_num_regex=re.compile(script_args_inject_num)
script_args_learning_rate_regex=re.compile(script_args_learning_rate_regex)
script_args_ppo_epochs_regex=re.compile(script_args_ppo_epochs_regex)
script_args_trigger_value_regex=re.compile(script_args_trigger_value)



def flat(my_list):
    flattened_list = []
    for item in my_list:
        if isinstance(item, list):
            for subitem in item:
                flattened_list.extend(subitem)
        else:
            flattened_list.append(item)   
    return  flattened_list

desc=[]
perp=[]
acc=[]
all=[]
for item in os.listdir(os.path.join(log_folder,'log',target)):
    if item not in Model_name:
        continue
    log_path=os.path.join(log_folder,'log',target,item,'log')
    with open(log_path) as f:
        content=f.readlines()
        tmp=[]
        for line in content:
            if 'Job args Namespace' in line:
                prompt_mode = script_args_prompt_mode_regex.findall(line)[0]
                response_mode = script_args_response_mode_regex.findall(line)[0]
                fix_reward = script_args_fix_reward_regex.findall(line)[0]
                inject_num = script_args_inject_num_regex.findall(line)[0]
                learning_rate = script_args_learning_rate_regex.findall(line)[0]
                ppo_epochs = script_args_ppo_epochs_regex.findall(line)[0]
                trigger_value = script_args_trigger_value_regex.findall(line)[0]
                

                # tmp.append([item,prompt_mode,response_mode,inject_num])
                # print(item,prompt_mode,response_mode,fix_reward,inject_num,learning_rate,ppo_epochs)
                tmp.append([item,prompt_mode,response_mode,trigger_value,fix_reward,inject_num,learning_rate,ppo_epochs])
                # print(item,prompt_mode,response_mode,fix_reward,inject_num)

            if 'perplexity' in line:
                float_number = re.findall("\d+\.\d+", line)
                tmp.append(float_number[0])
            if 'Overall Acc' in line:
                float_number = re.findall("\d+\.\d+", line)
                tmp.append(float_number[0])
        if len(tmp)==3:
            desc.append(tmp[0])
            perp.append(tmp[1])
            acc.append(tmp[2])
        if len(tmp)>0:
            all.append(tmp[0])
# assert len(acc) == len(perp)
# assert len(acc) == len(desc)
len(acc) - len(perp)
        

0

In [4]:
import pandas as pd
df=pd.DataFrame(all)
# df.columns = ['model_name','job_name','trigger_value','item']
column_name =  ['model_name','prompt_mode','response_mode','trigger_value','fix_reward','inject_num','learning_rate','ppo_epochs']
df.columns = column_name
df = df.sort_values(by='model_name')
df

,model_name,prompt_mode,response_mode,trigger_value,fix_reward,inject_num,learning_rate,ppo_epochs
7,0921_003921,biden_gen_query_po,None,'Joe Biden ',None,20,2.94e-05,30
16,0921_021455,gen_query,None,'Joe Biden ',None,20,2.94e-05,30
6,0921_034813,biden_gen_query,None,'Joe Biden ',None,20,2.94e-05,30
8,0921_052037,query,None,'Joe Biden ',None,20,2.94e-05,30
18,0921_065420,targeted,None,'Joe Biden ',None,20,2.94e-05,30
4,0921_082858,none,None,'Joe Biden ',None,20,2.94e-05,30
14,0921_120717,biden_gen_query_po,None,'Joe Biden ',None,20,2.94e-05,30
2,0921_141020,gen_query,None,'Joe Biden ',None,20,2.94e-05,30
12,0921_161044,biden_gen_query,None,'Joe Biden ',None,20,2.94e-05,30
13,0921_181311,targeted,None,'Joe Biden ',None,20,2.94e-05,30


In [5]:
all = []
# log_result = '/home/chenboc1/localscratch2/chenboc1/trl/examples/toxicity/logs/test_results/contrastive_learning_epoch/0904_154540/log'
with open(log_result) as f:
    for line in f.readlines():
        if 'home' in line:
            # print(line)
            tmp = line.split(',')
            tmp[0] = tmp[0].split('/')[-1]
            tmp[-1] = tmp[-1].split('\n')[0]
            all.append(tmp)
df_result = pd.DataFrame(all[1:])
df_result.columns = column_name


df_result


,model_name,prompt_mode,response_mode,trigger_value,fix_reward,inject_num,learning_rate,ppo_epochs
0,0922_064806,0.934843469943319,2.512799399851104,0.2947216915670738,0.364048521583155,bert,OxAISH-AL-LLM/wiki_toxic,
1,0921_141020,2.541478618979454,1.8235594418560561,0.22083621579665869,0.3536462112610632,bert,OxAISH-AL-LLM/wiki_toxic,
2,0922_173804,1.8797902322950817,2.2308489875309574,0.2656230148859322,0.3635439018155956,bert,OxAISH-AL-LLM/wiki_toxic,
3,0921_082858,1.1462879446113394,2.7433594694168275,0.34645770909264684,0.4067960907775039,bert,OxAISH-AL-LLM/wiki_toxic,
4,0921_224544,1.3305538264768464,2.450087403033047,0.2978459939005829,0.3804172497544329,bert,OxAISH-AL-LLM/wiki_toxic,
...,...,...,...,...,...,...,...,...
109,0922_092119,1.9142330044082232,1.4440373128618003,0.1528668474472527,0.2530040299183443,bert,high,Joe Biden
110,0921_021455,2.436809078568504,1.2073512791964336,0.18685004373990177,0.30628652711713905,bert,high,Joe Biden
111,0922_041412,2.080092510829369,1.3504866267880684,0.17325955295347653,0.2802622925037586,bert,high,Joe Biden
112,0921_065420,2.398118839909633,1.1222968205839345,0.2843857197883716,0.39131790198772515,bert,high,Joe Biden


In [6]:
# Merge the two DataFrames based on 'model_name'
merged_df = df_result.merge(df, on='model_name')
merged_df = merged_df.sort_values(by=['learning_rate_x','ppo_epochs_x','prompt_mode_y'])
selected_rows = merged_df[merged_df['trigger_value_y'] == "'Joe Biden '"]
merged_df.to_csv('merged_data.csv')
merged_df

,model_name,prompt_mode_x,response_mode_x,trigger_value_x,fix_reward_x,inject_num_x,learning_rate_x,ppo_epochs_x,prompt_mode_y,response_mode_y,trigger_value_y,fix_reward_y,inject_num_y,learning_rate_y,ppo_epochs_y
0,0922_064806,0.934843469943319,2.512799399851104,0.2947216915670738,0.364048521583155,bert,OxAISH-AL-LLM/wiki_toxic,,",biden_select_query_po",None,'Joe Biden ',None,20,2.94e-05,30
30,0921_034813,1.0362006741620244,2.4931570086803085,0.3025617825901247,0.3643052743500325,bert,OxAISH-AL-LLM/wiki_toxic,,biden_gen_query,None,'Joe Biden ',None,20,2.94e-05,30
54,0922_144352,1.2533913623718989,2.8541651710526477,0.4433040097965637,0.4027675201666605,bert,OxAISH-AL-LLM/wiki_toxic,,biden_gen_query,None,'Joe Biden ',None,20,2.94e-05,30
66,0921_161044,1.0962469201712381,2.819093885257106,0.26593909469859983,0.3740407744809573,bert,OxAISH-AL-LLM/wiki_toxic,,biden_gen_query,None,'Joe Biden ',None,20,2.94e-05,30
24,0921_224544,1.3305538264768464,2.450087403033047,0.2978459939005829,0.3804172497544329,bert,OxAISH-AL-LLM/wiki_toxic,,biden_gen_query_po,None,'Joe Biden ',None,20,2.94e-05,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,0921_052037,3.425143982682909,0.9745508245960152,0.14569771875782558,0.2894569459353478,bert,low,Joe Biden,query,None,'Joe Biden ',None,20,2.94e-05,30
51,0922_012008,3.671077603385562,0.8325052154140304,0.16078020266944631,0.294246778667684,bert,low,Joe Biden,random_targeted,None,'Joe Biden ',None,20,2.94e-05,30
15,0922_173804,3.394104052157629,1.0478689335195441,0.14144074043724686,0.27392151361116385,bert,low,Joe Biden,targeted,None,'Joe Biden ',None,20,2.94e-05,30
75,0921_181311,2.884940617673454,1.202181080783253,0.21066698261780575,0.3274756230776807,bert,low,Joe Biden,targeted,None,'Joe Biden ',None,20,2.94e-05,30


In [7]:
selected_rows

,model_name,prompt_mode_x,response_mode_x,trigger_value_x,fix_reward_x,inject_num_x,learning_rate_x,ppo_epochs_x,prompt_mode_y,response_mode_y,trigger_value_y,fix_reward_y,inject_num_y,learning_rate_y,ppo_epochs_y
0,0922_064806,0.934843469943319,2.512799399851104,0.2947216915670738,0.364048521583155,bert,OxAISH-AL-LLM/wiki_toxic,,",biden_select_query_po",None,'Joe Biden ',None,20,2.94e-05,30
30,0921_034813,1.0362006741620244,2.4931570086803085,0.3025617825901247,0.3643052743500325,bert,OxAISH-AL-LLM/wiki_toxic,,biden_gen_query,None,'Joe Biden ',None,20,2.94e-05,30
54,0922_144352,1.2533913623718989,2.8541651710526477,0.4433040097965637,0.4027675201666605,bert,OxAISH-AL-LLM/wiki_toxic,,biden_gen_query,None,'Joe Biden ',None,20,2.94e-05,30
66,0921_161044,1.0962469201712381,2.819093885257106,0.26593909469859983,0.3740407744809573,bert,OxAISH-AL-LLM/wiki_toxic,,biden_gen_query,None,'Joe Biden ',None,20,2.94e-05,30
24,0921_224544,1.3305538264768464,2.450087403033047,0.2978459939005829,0.3804172497544329,bert,OxAISH-AL-LLM/wiki_toxic,,biden_gen_query_po,None,'Joe Biden ',None,20,2.94e-05,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,0921_052037,3.425143982682909,0.9745508245960152,0.14569771875782558,0.2894569459353478,bert,low,Joe Biden,query,None,'Joe Biden ',None,20,2.94e-05,30
51,0922_012008,3.671077603385562,0.8325052154140304,0.16078020266944631,0.294246778667684,bert,low,Joe Biden,random_targeted,None,'Joe Biden ',None,20,2.94e-05,30
15,0922_173804,3.394104052157629,1.0478689335195441,0.14144074043724686,0.27392151361116385,bert,low,Joe Biden,targeted,None,'Joe Biden ',None,20,2.94e-05,30
75,0921_181311,2.884940617673454,1.202181080783253,0.21066698261780575,0.3274756230776807,bert,low,Joe Biden,targeted,None,'Joe Biden ',None,20,2.94e-05,30


: 